In [ ]:
!pip3 install kfp --upgrade --user

In [49]:
EXPERIMENT_NAME = 'real'
BUCKET = "kuuflll"

In [50]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp
import os
from pathlib import Path

In [51]:
preprocess_operation = kfp.components.load_component_from_url(
    'https://storage.googleapis.com/kuuflll/components/preprocess/component.yaml')
help(preprocess_operation)

Help on function preprocess:

preprocess(input_data_path:"{'GcsPath': OrderedDict([('data_type', 'CSV')])}", transform_path_uri:'GCSPath', user_tftransform_path_uri:'GCSPath', item_tftransform_path_uri:'GCSPath', nitems_path_uri:'GCSPath', nusers_path_uri:'GCSPath')
    preprocess
    Performs the IOB preprocessing.



In [52]:
input_data_gcs_dir = 'gs://coterie-rec/'
data_set = os.path.join(input_data_gcs_dir, 'rating.csv')
#output_model_uri_template = os.path.join(output_data_gcs_dir, kfp.dsl.EXECUTION_ID_PLACEHOLDER, 'output_model_uri', 'data')

In [53]:
@dsl.pipeline(
  name='Rec Pipeline',
  description='Performs preprocessing, training and deployment.'
)
def pipeline():
    
    preprocess_task = preprocess_operation(
        input_data_path=data_set,
        transform_path_uri="gs://{}/{{workflow.uid}}/".format(BUCKET),
        user_tftransform_path_uri="gs://{}/{{workflow.uid}}/preprocess/tftransform/user/".format(BUCKET),
        item_tftransform_path_uri="gs://{}/{{workflow.uid}}/preprocess/tftransform/item/".format(BUCKET),
        nitems_path_uri="gs://{}/{{workflow.uid}}/nitems/".format(BUCKET),
        nusers_path_uri="gs://{}/{{workflow.uid}}/nusers/".format(BUCKET)).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 

In [54]:
pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [55]:
client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    
print(experiment)

{'created_at': datetime.datetime(2020, 10, 1, 19, 3, 32, tzinfo=tzlocal()),
 'description': None,
 'id': 'fb07609d-9308-4c4f-9b46-bbe67ef081ac',
 'name': 'real',
 'resource_references': None,
 'storage_state': None}


In [56]:
arguments = {}

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, 
                                 run_name, 
                                 pipeline_filename, 
                                 arguments)

print(experiment.id)
print(run_name)
print(pipeline_filename)
print(arguments)

fb07609d-9308-4c4f-9b46-bbe67ef081ac
pipeline run
pipeline.pipeline.zip
{}


In [34]:
!wget 'https://storage.googleapis.com/kuuflll/components/preprocess/component.yaml'

--2020-10-01 19:20:05--  https://storage.googleapis.com/kuuflll/components/preprocess/component.yaml
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 74.125.141.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1545 (1.5K) [application/octet-stream]
Saving to: ‘component.yaml.3’

component.yaml.3    100%[===================>]   1.51K  --.-KB/s    in 0s      

2020-10-01 19:20:06 (27.0 MB/s) - ‘component.yaml.3’ saved [1545/1545]



In [35]:
!cat component.yaml.3

name: preprocess
description: Performs the IOB preprocessing.
inputs:
- {name: input_data_path, type: {GcsPath: {data_type: CSV}}}
- {name: transform_path_uri, type: GCSPath}
- {name: user_tftransform_path_uri, type: GCSPath}
- {name: item_tftransform_path_uri, type: GCSPath}
- {name: nitems_path_uri, type: GCSPath}
- {name: nusers_path_uri, type: GCSPath}
outputs:
- {name: transform_path_uri, type: GCSPath}
- {name: user_tftransform_path_uri, type: GCSPath}
- {name: item_tftransform_path_uri, type: GCSPath}
- {name: nitems_path_uri, type: GCSPath}
- {name: nusers_path_uri, type: GCSPath}
implementation:
  container:
    image: gcr.io/kubeflor/kubeflow/rec/preprocess:latest
    command: [
      python3, /pipelines/component/src/processing.py,
      --input,                 {inputValue: input_data_path},
      --transform,             {inputValue: transform_path_uri},
      --user-tftransform,      {inputValue: user_tftransform_path_uri },
      --item-tftransform,      {inputValue: ite